In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [12]:
import imagedataset as imgdat
import vlad

In [13]:
#Get Annotations
annotations = imgdat.GetAnnotations()

In [14]:
#Try to load all descriptors previously generated if not generate one
try:
    all_descriptors = imgdat.LoadAllDescriptors()
except:
    all_descriptors = imgdat.GenerateAllDescriptors(annotations)

Loading all sift descriptors in dataset from file...
All descriptors loaded ! 


In [15]:
#Use standardscaler for all SIFT descriptors before process them with any representation algorithm.
sc_dscs = StandardScaler()
all_descriptors = sc_dscs.fit_transform(all_descriptors)

In [16]:
#Clustering descriptors using KMeans in 120 classes 
try:
    kmeans_vlad = vlad.LoadKmeans()
except:
    n_clusters = 120
    kmeans_vlad = vlad.GenerateKmeans(all_descriptors, n_clusters)

Loading kmeans...


In [17]:
#Image representation
try:
    X_vlad = np.load("vlad_X.npy")
    y_vlad = np.load("vlad_y.npy")
except:
    X_vlad, y_vlad = vlad.VladFeatures(annotations, kmeans_vlad, sc_dscs)
    np.save("vlad_X", X_vlad)
    np.save("vlad_y", y_vlad)

In [18]:
#Classification 
#Dataset splitting into the training set and test set
X_train_vlad, X_test_vlad, y_train_vlad, y_test_vlad = train_test_split(X_vlad, y_vlad, test_size = 0.2, random_state = 0)

In [19]:
#Scaling representation features
sc_repr_vlad = StandardScaler()
X_train_vlad = sc_repr_vlad.fit_transform(X_train_vlad)
X_test_vlad = sc_repr_vlad.transform(X_test_vlad)

In [21]:
#Using Gaussian Naive Bayes
clf_nb = GaussianNB()
clf_nb.fit(X_train_vlad, y_train_vlad)
print('==========')
print('train score:' + str(clf_nb.score(X_train_vlad,y_train_vlad)))
print('test score:' + str(clf_nb.score(X_test_vlad,y_test_vlad)))
y_pred = clf_nb.predict(X_test_vlad)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test_vlad, y_pred)
print(cm)

train score:0.442748091603
test score:0.434343434343
[[33  5]
 [51 10]]
